In [9]:
library(caret)
library(randomForest)
library(doParallel)


 要求されたパッケージ foreach をロード中です 

 要求されたパッケージ iterators をロード中です 

 要求されたパッケージ parallel をロード中です 



In [8]:
install.packages('doParallel')


 ダウンロードされたパッケージは、以下にあります 
 	/var/folders/p0/xr8c62gn79qghlvd5xp31xr40000gn/T//RtmpBgwUXv/downloaded_packages 


In [1]:
data_filtered <- read.csv('data_filtered.csv')
df_cleaned <- read.csv('df_cleaned.csv')

In [2]:

# Convert Churn column to binary numeric values
data_filtered$Churn <- ifelse(data_filtered$Churn == 1, 1, 0)

data_filtered$customerID <- df_cleaned$customerID

# Convert all character columns to factors
data_filtered[] <- lapply(data_filtered, function(x) {
  if (is.character(x)) {
    as.factor(x)
  } else {
    x
  }
})

# Split data into training and testing sets
set.seed(42)
train_index <- sample(1:nrow(data_filtered), 0.7 * nrow(data_filtered))
train_data <- data_filtered[train_index, ]
test_data <- data_filtered[-train_index, ]

In [13]:

registerDoParallel(cores = 4) 
train_data$Churn <- as.factor(train_data$Churn)
test_data$Churn <- as.factor(test_data$Churn)
train_control <- trainControl(method = "cv", number = 5, allowParallel = TRUE)

In [14]:
model_churn <- train(Churn ~ . - customerID, data = train_data, method = "rf", ntree = 50, trControl = train_control)
model_tenure <- train(tenure ~ . - customerID - Churn, data = train_data, method = "rf", ntree = 50, trControl = train_control)


In [15]:
# Make predictions
predictions_churn <- predict(model_churn, test_data)
predictions_tenure <- predict(model_tenure, test_data)

# Show predictions
cat("Predicted Churn:\n")
print(predictions_churn)
cat("Predicted Tenure:\n")
print(predictions_tenure)


Predicted Churn:
   [1] 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
  [38] 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1
  [75] 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1
 [112] 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1
 [149] 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 0
 [186] 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 [223] 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1
 [260] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 [297] 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1
 [334] 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 [371] 1 1 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1
 [408] 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0
 [445] 1 1 

In [17]:
# Calculate performance metrics
churn_accuracy <- mean(predictions_churn == test_data$Churn)
tenure_mse <- mean((predictions_tenure - test_data$tenure)^2)

# Print results
cat("Churn accuracy:", churn_accuracy, "\n")
cat("Tenure MSE:", tenure_mse, "\n")

Churn accuracy: 0.8592417 
Tenure MSE: 230.5673 
